In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Cargar el dataset
df = pd.read_csv('DATASET\Movies_EDA_ILB.csv', encoding='utf-8')

In [3]:
df.head(1)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,return,release_year,genres_name,idgenres,iso_spokelang,namescast,idcast,directors
0,30.0,862,en,"Led by Woody, Andys toys live happily in his r...",21.946943,1995-10-30,373.554033,81.0,Released,Toy Story,7.7,5415.0,0.000012,1995,"Animation, Comedy, Family","16, 35, 10751",en,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","31, 12898, 7167, 12899, 12900, 7907, 8873, 111...",John Lasseter


In [4]:
df.shape

(40946, 20)

In [5]:
df.drop_duplicates(subset=['title'],inplace=True)
df=df.reset_index(drop=True)
df.shape

(38114, 20)

In [6]:
model_df=df[['overview','title','genres_name','namescast','directors']]

In [7]:
model_df.head(3)

,overview,title,genres_name,namescast,directors
0,"Led by Woody, Andys toys live happily in his r...",Toy Story,"Animation, Comedy, Family","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,When siblings Judy and Peter discover an encha...,Jumanji,"Adventure, Fantasy, Family","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,A family wedding reignites the ancient feud be...,Grumpier Old Men,"Romance, Comedy","Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch


In [8]:
model_df['overview']=model_df['overview'].map(str)
model_df['title']=model_df['title'].map(str)
model_df['genres_name']=model_df['genres_name'].map(str)
model_df['namescast']=model_df['namescast'].map(str)
model_df['directors']=model_df['directors'].map(str)

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_10476\4170630534.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['overview']=model_df['overview'].map(str)
C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_10476\4170630534.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['title']=model_df['title'].map(str)
C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_10476\4170630534.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [9]:
for col in ['overview','title','genres_name','namescast','directors']:
    for val in ['[',']','\'']:
        model_df[col]=model_df[col].str.replace(val,'')
    model_df[col]=model_df[col].astype(str)

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_10476\2861905056.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  model_df[col]=model_df[col].str.replace(val,'')
C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_10476\2861905056.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[col]=model_df[col].str.replace(val,'')
C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_10476\2861905056.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [10]:
model_df.head(3)

,overview,title,genres_name,namescast,directors
0,"Led by Woody, Andys toys live happily in his r...",Toy Story,"Animation, Comedy, Family","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,When siblings Judy and Peter discover an encha...,Jumanji,"Adventure, Fantasy, Family","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,A family wedding reignites the ancient feud be...,Grumpier Old Men,"Romance, Comedy","Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch


In [11]:
def convert_columns_to_lowercase(df):
    for column in df.select_dtypes(include='object'):
        df[column] = df[column].str.lower()
    return df

In [12]:
model_df = convert_columns_to_lowercase(model_df)

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_10476\2484958803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.lower()


In [13]:
def create_text(model_data):
    text = model_data['title']
    for column in ['genres_name', 'overview', 'namescast', 'directors']:
        if pd.notnull(model_data[column]) and isinstance(model_data[column], str):
            text += ' ' + model_data[column].lower()
    return text

In [14]:
model_df.head(3)

,overview,title,genres_name,namescast,directors
0,"led by woody, andys toys live happily in his r...",toy story,"animation, comedy, family","tom hanks, tim allen, don rickles, jim varney,...",john lasseter
1,when siblings judy and peter discover an encha...,jumanji,"adventure, fantasy, family","robin williams, jonathan hyde, kirsten dunst, ...",joe johnston
2,a family wedding reignites the ancient feud be...,grumpier old men,"romance, comedy","walter matthau, jack lemmon, ann-margret, soph...",howard deutch


In [15]:
model_df['texto'] = model_df['title'] + ' ' + model_df['genres_name'] + ' ' + model_df['overview'] + ' ' + model_df['namescast'] + ' ' + model_df['directors']

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_10476\1557276399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['texto'] = model_df['title'] + ' ' + model_df['genres_name'] + ' ' + model_df['overview'] + ' ' + model_df['namescast'] + ' ' + model_df['directors']


In [16]:
def remove_commas(text):
    if isinstance(text, str):
        return text.replace(',', '')
    else:
        return ''

In [17]:
# Aplicar la función clean_keywords a la columna 'keywords'
model_df['texto'] = model_df['texto'].apply(remove_commas)

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_10476\361630095.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['texto'] = model_df['texto'].apply(remove_commas)


In [18]:
model_df.head(3)

,overview,title,genres_name,namescast,directors,texto
0,"led by woody, andys toys live happily in his r...",toy story,"animation, comedy, family","tom hanks, tim allen, don rickles, jim varney,...",john lasseter,toy story animation comedy family led by woody...
1,when siblings judy and peter discover an encha...,jumanji,"adventure, fantasy, family","robin williams, jonathan hyde, kirsten dunst, ...",joe johnston,jumanji adventure fantasy family when siblings...
2,a family wedding reignites the ancient feud be...,grumpier old men,"romance, comedy","walter matthau, jack lemmon, ann-margret, soph...",howard deutch,grumpier old men romance comedy a family weddi...


In [19]:
model_df=model_df.reset_index(drop=True)

In [20]:
"""def create_text(model_data):
    text = model_data['title']
    for i in model_data[1:]:
        text = text + ' ' + str(i.replace(',',' '))
    return text"""

"def create_text(model_data):\n    text = model_data['title']\n    for i in model_data[1:]:\n        text = text + ' ' + str(i.replace(',',' '))\n    return text"

In [21]:
"""model_df['texto']=model_df.apply(create_text,axis=1)"""

"model_df['texto']=model_df.apply(create_text,axis=1)"

In [22]:
model_df.to_csv('DATASET\Movies_prueba_ILB.csv',index=False)

In [23]:
model_df.drop(columns=['overview','genres_name','namescast','directors'], inplace=True)

In [24]:
model_df.head(3)

,title,texto
0,toy story,toy story animation comedy family led by woody...
1,jumanji,jumanji adventure fantasy family when siblings...
2,grumpier old men,grumpier old men romance comedy a family weddi...


In [25]:
model_df=model_df.reset_index(drop=True)

In [26]:
# Crear una instancia del CountVectorizer con los parámetros deseados
cv = CountVectorizer(stop_words='english', max_features=10000000000000000, min_df=1, max_df=20)  # Ajusta el valor de max_features según tus necesidades

In [27]:
# Definir el tamaño de lote para el cálculo de similitud
batch_size = 8000

In [28]:
# Obtener el número total de muestras y calcular el número de lotes
n_samples = len(model_df)
n_batches = (n_samples // batch_size) + 1

cosine_sim_list = []

for i in range(n_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, n_samples)

    # Obtener el subconjunto de datos
    subset_data = model_df.iloc[start:end]

    # Ajustar y transformar el CountVectorizer en el subconjunto de datos
    cv_matrix = cv.fit_transform(subset_data['texto'])

    # Calcular la similitud coseno entre los textos
    similarity_matrix = cosine_similarity(cv_matrix, cv_matrix)

    # Agregar la matriz de similitud a la lista
    cosine_sim_list.append(similarity_matrix)

In [29]:
# Encontrar la dimensión mínima a lo largo del eje 0
min_dim_0 = min(matrix.shape[0] for matrix in cosine_sim_list)
min_dim_1 = min(matrix.shape[1] for matrix in cosine_sim_list)

In [30]:
# Ajustar las dimensiones de las matrices al número mínimo de filas y columnas
cosine_sim_list = [matrix[:min_dim_1, :min_dim_0] for matrix in cosine_sim_list]

In [31]:
# Combinar las matrices de similitud coseno en una única matriz
cosine_sim = np.concatenate(cosine_sim_list, axis=0)

In [32]:
def recomendaciones(titulo, cosine_sim=cosine_sim):
    try:
        # Getting the index of the movie that matches the title
        idx = model_df[model_df['title'] == str(titulo).lower()].index[0]
        # Getting the similarity scores
        sim_scores = list(enumerate(cosine_sim[idx]))
        #Sorting the movies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Getting the top 5 recommendations
        sim_scores = sim_scores[1:6]
        movie_indices = [i[0] for i in sim_scores]
        recommendations=list(model_df['title'].iloc[movie_indices].str.title())
        return {'lista recomendada': recommendations} 
    except:
        return {'lista recomendada': ['LEGO DC Super Hero Girls: Brain Drain', 'The Lion of Thebes', 'Criminal Justice', 'Mad at the Moon', 'The Far Pavilions']}

In [37]:
recomendaciones('top gun')

{'lista recomendada': ['The Monkeys Mask',
  'Paid In Full',
  'At First Sight',
  'Ace Ventura: When Nature Calls',
  'Fly Away Home']}

In [34]:
model_df.head(50)

,title,texto
0,toy story,toy story animation comedy family led by woody...
1,jumanji,jumanji adventure fantasy family when siblings...
2,grumpier old men,grumpier old men romance comedy a family weddi...
3,waiting to exhale,waiting to exhale comedy drama romance cheated...
4,father of the bride part ii,father of the bride part ii comedy just when g...
5,heat,heat action crime drama thriller obsessive mas...
6,sabrina,sabrina comedy romance an ugly duckling having...
7,tom and huck,tom and huck action adventure drama family a m...
8,sudden death,sudden death action adventure thriller interna...
9,goldeneye,goldeneye adventure action thriller james bond...
